In [181]:
import spacy
from spacy.tokens import Doc, Token, Span
nlp = spacy.load("en_core_web_md")

In [182]:
from spacy.pipeline import EntityRuler

In [183]:
from spacy import displacy

In [184]:
import numpy as np

In [185]:
Doc.set_extension('ethnicity', default='',force=True)
Doc.set_extension('date', default='',force=True)

In [186]:
def getEthnicity(doc):
#     print(doc.ents)
    for ent in doc.ents:
#         print(ent.label_)
        if (ent.label_ == "NORP"):
            doc._.ethnicity = ent.text
    return doc

In [187]:
def getDate(doc):
#     print(doc.ents)
    dateList = []
    for ind, ent in enumerate(doc.ents):
#         print(ent.label_)
        if (ent.label_ == "DATE" or ent.label_ == 'AGE'):
            if  doc.ents[ind - 1].label_ in ['ABOVE','BELOW']:
                date = doc.ents[ind - 1].ent_id_ + " " + ent.text
                dateList.append(date)
            else:
                date = ent.text    
                dateList.append(date)
#     for ind, ent in enumerate(doc.ents):
#         if (ent.label_ == 'AGE_LABEL'):
#             if doc.ents[ind + 1].label_ in ['ABOVE','BELOW']:
#                 if doc.ents[ind + 2].text.is_digit:
#                     date =  doc.ents[ind + 1].ent_id_ + " " + doc.ents[ind + 2].ent_id_
    if len(dateList) > 0:
        doc._.date = dateList
    return doc

In [188]:
doc2 = nlp(" 'Age' is in range (XX-XX years/months)")

In [189]:
import pandas as pd

In [190]:
data = pd.read_csv('therapyAdded.csv',sep='|')

In [191]:
inc_changed = data.inclusion.str.replace('≥'," ≥ ")
inc_changed = inc_changed.str.replace('≤'," ≤ ")
inc_changed = inc_changed.str.replace('<'," < ")
inc_changed = inc_changed.str.replace('>'," > ")
# inc_changed = inc_changed.str.replace('-'," - ")
inc_changed = inc_changed.str.replace('='," = ")
inc_changed = inc_changed.str.replace('> =',">=")
inc_changed = inc_changed.str.replace('= >',"=>")
inc_changed = inc_changed.str.replace('< =',"<=")
inc_changed = inc_changed.str.replace('= <',"=<")


inc_changed = inc_changed.str.replace('⩾'," ⩾ ")
inc_changed = inc_changed.str.replace('year'," year")
inc_changed = [' '.join(text.split()) for text in inc_changed]

In [192]:
data['inclusion'] = inc_changed

In [193]:
genderPatterns = [{"label": "GENDER", "pattern": [{"LOWER": "male"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "men"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "males"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "man"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "boy"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "guy"}], "id":"male"},
                  {"label": "GENDER", "pattern": [{"LOWER": "female"}], "id":"female"},
                  {"label": "GENDER", "pattern": [{"LOWER": "women"}], "id":"female"},
                  {"label": "GENDER", "pattern": [{"LOWER": "females"}], "id":"female"},
                  {"label": "GENDER", "pattern": [{"LOWER": "girl"}], "id":"female"},
                  {"label": "GENDER", "pattern": [{"LOWER": "girls"}], "id":"female"},
                  {"label": "GENDER", "pattern": [{"LOWER": "woman"}], "id":"female"}]

In [194]:
    agePatterns = [{"label": "RANGE", "pattern": [{"LOWER": "aged"},{"LOWER": "between", "OP":"?"},{"IS_DIGIT": True},{"TEXT": "-"},{"IS_DIGIT": True} ], "id":"between"},
                   {"label": "AGE_LABEL", "pattern": [{"LOWER": "age"}], "id":"age"},
                   {"label": "AGE_LABEL", "pattern": [{"LOWER": "aged"}], "id":"age"},
                   {"label": "AGE_LABEL", "pattern": [{"LOWER": "ages"}], "id":"age"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "years"}], "id":"year"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "year"}], "id":"year"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "months"}], "id":"month"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "month"}], "id":"month"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "weeks"}], "id":"week"},
                   {"label": "DATE_LABEL", "pattern": [{"LOWER": "week"}], "id":"week"},
                   {"label": "AGE_LABEL", "pattern": [{"LOWER": "old"}], "id":"old"},
]

In [195]:
comparePatterns = [
    {"label": "ABOVE", "pattern": [{"TEXT": "≥"}], "id":"GE"},
    {"label": "ABOVE", "pattern": [{"TEXT": "⩾"}], "id":"GE"},
    {"label": "ABOVE", "pattern": [{"TEXT": ">"}], "id":"G"},
    {"label": "ABOVE", "pattern": [{"TEXT": ">"},{"TEXT": "="}], "id":"GE"},
    {"label": "ABOVE", "pattern": [{"TEXT": "="},{"TEXT": ">"}], "id":"GE"},
    {"label": "BELOW", "pattern": [{"TEXT": "<"},{"TEXT": "="}], "id":"LE"},
    {"label": "BELOW", "pattern": [{"TEXT": "<"}], "id":"L"},
    {"label": "BELOW", "pattern": [{"TEXT": "≤"}], "id":"LE"}    
]

In [196]:
ruler = EntityRuler(nlp)
ruler.add_patterns(genderPatterns)
ruler.add_patterns(agePatterns)
ruler.add_patterns(comparePatterns)
nlp.add_pipe(ruler)

In [197]:
nlp.add_pipe(getEthnicity, last=True)
nlp.add_pipe(getDate, after='getEthnicity')

In [198]:
nlp.pipe_names

['tagger', 'parser', 'ner', 'entity_ruler', 'getEthnicity', 'getDate']

In [199]:
docs = list(nlp.pipe(data.inclusion)) 

In [200]:
data['ageSimilarity'] = [doc.similarity(doc2) for doc in docs]

D:\Users\Prem2282@gmail.com\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  """Entry point for launching an IPython kernel.


In [201]:
#data[['inclusion','similarity']].sort_values('similarity').head(20)

In [202]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,inclusion,nct_id,Topic,hasDiagnosis,hasTherapy,hasTreatment,TherapyList,disease,chemical,sentenceCount,ageSimilarity
0,0,14633,4245,Inclusion Criteria:,NCT04425031,0,False,False,False,NaN,[],[],1,0.377586
1,1,4112,1150,Inclusion Criteria:,NCT04470856,0,False,False,False,NaN,[],[],1,0.377586
2,2,12543,3623,Inclusion Criteria:,NCT04434417,0,False,False,False,NaN,[],[],1,0.377586
3,3,6767,1896,Inclusion Criteria: - Patient after an arthros...,NCT04460001,0,False,False,False,NaN,[],[],1,0.684108
4,4,4106,1146,Inclusion Criteria: - Contact person (defined ...,NCT04470921,0,False,False,False,NaN,[],[],1,0.711641


In [203]:
def getAgeKeyword(doc):
    ageKeyword = []
    for token in doc:
        string = token.text.lower()
        substring_list = ['age','year','old','month']
        for substring in substring_list:
            if string.find(substring) >= 0:
#                 print(substring)
                ageKeyword.append(substring)
            
#         ageKeyword = [substring for substring in substring_list if string.find(substring) >= 0]
    return ageKeyword

In [204]:
def getGender(doc):
    gender = 'all'
    male_list = ['male','men','man','boy']
    female_list = ['female','women','men','girl']
    is_male = any([token.text.lower() in male_list for token in doc])
    is_female = any([token.text.lower() in female_list for token in doc])
#     print(doc)
#     print(is_male)
#     print(is_female)
    
#     for token in doc:
#         string = token.text.lower()
#         male_list = ['male','men','man','boy']
#         female_list = ['female','women','men','girl']
#         is_male = string in male_list
#         is_female = string in female_list
#         print(doc)
#         print(is_male)
#         print(is_female)
    if is_male & is_female:
        gender = 'all'
    elif is_male:
        gender = 'male'
    elif is_female:
        gender = 'female'
    else:
        gender = 'all'

    return gender

In [205]:
# getAgeKeyword(docs[0])

data['AgeKeyword'] = [getAgeKeyword(doc) for doc in docs]

In [206]:
data['gender'] = [getGender(doc) for doc in docs]

In [207]:
getGender(docs[35])

'all'

In [208]:
data.groupby('gender').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,inclusion,nct_id,Topic,hasDiagnosis,hasTherapy,hasTreatment,TherapyList,disease,chemical,sentenceCount,ageSimilarity,AgeKeyword
gender,,,,,,,,,,,,,,,
all,16362,16362,16362,16362,16362,16362,16362,16362,16362,1046,16362,16362,16362,16362,16362
female,671,671,671,671,671,671,671,671,671,64,671,671,671,671,671
male,170,170,170,170,170,170,170,170,170,8,170,170,170,170,170


In [209]:
data[data['gender'] == 'female']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,inclusion,nct_id,Topic,hasDiagnosis,hasTherapy,hasTreatment,TherapyList,disease,chemical,sentenceCount,ageSimilarity,AgeKeyword,gender
265,265,7480,2087,Inclusion Criteria: - Women aged 18 to 38 years.,NCT04457206,0,False,False,False,NaN,[],[],1,0.770364,"[age, year]",female
356,356,7030,1972,All other female subjects must have the reason...,NCT04458740,0,False,False,False,NaN,[],[],1,0.711633,[year],female
403,403,10948,3099,Inclusion Criteria: - Having 18 years or more;...,NCT04441996,0,True,False,False,NaN,"['Depression', 'DSM-5']",[],1,0.754970,[year],female
990,990,9906,2796,Group B COPD CAT: ≥ 10 or mMRC: ≥ 2 Exacerbati...,NCT04446780,0,False,False,False,NaN,"['COPD', 'COPD', '≥2', '≥2']","['D COPD', 'smoking']",1,0.753399,[year],female
1336,1336,10667,3017,Inclusion criteria : Patients (consecutive inc...,NCT04443387,0,False,False,False,NaN,['Chalons'],"['CHG', 'CHG']",1,0.658078,[],female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16652,16652,14558,4228,- Patients with at least one evaluable or meas...,NCT04425291,38,False,True,False,"['chemotherapy', 'therapy']",['lung cancer'],[],1,0.724366,"[month, age]",female
16750,16750,8411,2349,13. Lactating female that is willing to refrai...,NCT04453371,38,False,False,False,NaN,[],[],2,0.728905,[],female
16974,16974,9743,2749,Their female partner of childbearing potential...,NCT04447456,39,False,False,False,NaN,[],['contraceptive'],1,0.590709,[],female
17020,17020,5893,1631,"Inclusion Criteria: - Healthy women, who have ...",NCT04463758,39,False,False,False,NaN,[],[],1,0.703911,[month],female


In [210]:
# for doc in docs:
#     displacy.render(doc, style="ent")

In [211]:
spacy.explain('NORP')

'Nationalities or religious or political groups'

In [212]:
data['numbers'] = [[ token.text for token in doc if token.is_digit ] for doc in docs ]

In [213]:
data.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,inclusion,nct_id,Topic,hasDiagnosis,hasTherapy,hasTreatment,TherapyList,disease,chemical,sentenceCount,ageSimilarity,AgeKeyword,gender,numbers
0,0,14633,4245,Inclusion Criteria:,NCT04425031,0,False,False,False,NaN,[],[],1,0.377586,[],all,[]
1,1,4112,1150,Inclusion Criteria:,NCT04470856,0,False,False,False,NaN,[],[],1,0.377586,[],all,[]
2,2,12543,3623,Inclusion Criteria:,NCT04434417,0,False,False,False,NaN,[],[],1,0.377586,[],all,[]
3,3,6767,1896,Inclusion Criteria: - Patient after an arthros...,NCT04460001,0,False,False,False,NaN,[],[],1,0.684108,[year],all,"[1, 4]"
4,4,4106,1146,Inclusion Criteria: - Contact person (defined ...,NCT04470921,0,False,False,False,NaN,[],[],1,0.711641,"[age, year, old]",all,[18]


In [214]:
# Import the Matcher library
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

matcher.remove('greater_than')
matcher.remove('between')

In [215]:

pattern1 = [{"TEXT": '≥'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern1a = [{"TEXT": '> ='},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern1b = [{"TEXT": '= >'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern1c = [{"TEXT": '='},{"TEXT": '>'},  {"LIKE_NUM": True} ]
pattern1d = [{"TEXT": '<'},{"TEXT": '='},  {"LIKE_NUM": True} ]
# pattern1a = [{"TEXT": '⩾'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern2 = [{"LIKE_NUM": True},{"TEXT": '-'},{"LIKE_NUM": True}]
pattern3 = [{"LIKE_NUM": True},{"LOWER": 'to'},{"LIKE_NUM": True}]
pattern4 =  [{"LIKE_NUM": True},{"LOWER": 'and'},{"LIKE_NUM": True}]
pattern5 = [{"TEXT": '>'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern6 = [{"TEXT": '<'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern7 = [{"TEXT": '≤'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern7a = [{"TEXT": '< ='},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern7b = [{"TEXT": '= >'},{"TEXT": ' ', "OP":"*"},  {"LIKE_NUM": True} ]
pattern7c = [{"TEXT": '='},{"TEXT": '>'},  {"LIKE_NUM": True} ]
pattern7d = [{"TEXT": '<'},{"TEXT": '='},  {"LIKE_NUM": True} ]
pattern8a = [{"LIKE_NUM": True},{"LOWER": 'years'}]
pattern8 = [{"LIKE_NUM": True},{"LOWER": 'years',"OP":"?"},{"LOWER": 'of',"OP":"?"},{"LOWER": 'age',"OP":"?"},{"LOWER": 'or'}]
pattern10 = [{"LIKE_NUM": True},{"LOWER": 'years',"OP":"?"},{"LOWER": 'of',"OP":"?"},{"LOWER": 'age',"OP":"?"},{"LOWER": 'and'}]
pattern9 = [{"LEMMA": "age","OP":"?"},{"LOWER": 'over'},{"LOWER": "the","OP":"?"},{"TEXT": "age","OP":"?"},{"TEXT": "of","OP":"?"},{"LIKE_NUM":True}]
pattern11 = [{"LEMMA": "age","OP":"?"},{"LOWER": 'under'},{"LIKE_NUM":True}]
pattern12 = [{"LEMMA": "age","OP":"?"},{"LOWER": '>'},{"LIKE_NUM":True}]
pattern13 = [{"LOWER": "aged"},{"LIKE_NUM":True}]
pattern14 = [{"LOWER": "at"},{"LOWER": "least"},{"LIKE_NUM":True}]
pattern15 = [{"LOWER": "older"},{"LOWER": "than"},{"LIKE_NUM":True}]
pattern16 = [{"LOWER": "above"},{"LIKE_NUM":True}]
pattern17 = [{"LOWER": "greater"},{"LOWER": "than"},{"LIKE_NUM":True}]
pattern18 = [{"LOWER": "more"},{"LOWER": "than"},{"LIKE_NUM":True}]
matcher.add('greater_than', None, pattern1, pattern1a, pattern1b, pattern1c, pattern1d)
matcher.add('between', None, pattern2)
matcher.add('between_to', None, pattern3)
matcher.add('between_and', None, pattern4)
matcher.add('greater_notequal', None, pattern5)
matcher.add('lesser_notequal', None, pattern6)
matcher.add('lesser_than', None, pattern7, pattern7a, pattern7b, pattern1c, pattern1d)
matcher.add('or_older', None, pattern8, pattern8a, pattern10)
matcher.add('over', None, pattern9)
matcher.add('under', None, pattern11)
matcher.add('age>', None, pattern12,pattern13,pattern14,pattern15,pattern16,pattern17,pattern18)

In [216]:
def getPattern(sentence):
    criteria = sentence
#     print(criteria)
    matches = matcher(criteria)
#     return(matches)
    result = ""
    lower = '0'
    upper = '0'
    
    for match_id, start,end in matches:
        match_pattern = nlp.vocab[match_id].text
        
        if match_pattern == 'greater_than':
            result_item = {
                'pattern': '>=',
                'lower': criteria[end-1]
            }
            lower = criteria[end-1]
            
        if match_pattern == 'greater_notequal':
            result_item = {
                'pattern': '>',
                'lower': criteria[end-1]
            }
            lower: criteria[end-1] - 1
        if match_pattern == 'lesser_notequal':
            result_item = {
                'pattern': '<',
                'upper': criteria[end-1]
            }
#             upper = criteria[end-1] - 1
            upper = criteria[end-1]
        if match_pattern == 'lesser_than':
            result_item = {
                'pattern': '<=',
                'upper': criteria[end-1]
            }
            upper =  criteria[end-1]
        if match_pattern in ['between','between_to','between_and']:
            result_item = {
                'pattern': '-',
                'lower': criteria[start],
                'upper': criteria[end-1],
            }
            lower = criteria[start]
            upper = criteria[end-1]

        if match_pattern in ['or_older']:
            result_item = {
                'pattern': '>=',
                'lower': criteria[start],
            }
            lower = criteria[start]
            
        if match_pattern in ['over']:
            result_item = {
                'pattern': '>=',
                'lower': criteria[end-1],
            }
            lower = criteria[end-1]
            
        if match_pattern in ['under']:
            result_item = {
                'pattern': '<',
                'upper': criteria[end-1],
            }
            upper = criteria[end-1]
            
        if match_pattern in ['age>']:
            result_item = {
                'pattern': '>',
                'lower': criteria[end-1],
            }
            lower = criteria[end-1]            
#         result.append(result_item)
        
        result = {
            'lower': str(lower),
            'upper': str(upper),
            'pattern': match_pattern,
            'phrase': criteria[start:end]
        }
    return result

In [217]:
#test
#sentence = data['inclusion'][74]
#    patternList = getPattern(sentence)
#    print(sentence)
#    print(patternList)

In [218]:
data['Pattern'] = [getPattern(doc) for doc in docs]

In [219]:
data['ethnicity'] = [doc._.ethnicity for doc in docs]

In [220]:
data['date'] = [doc._.date for doc in docs]

In [221]:
data.to_csv('demograpyAdded.csv',sep = '|')

In [222]:
similarity = data.ageSimilarity > 0.5
pattern = data.Pattern == ''
goodSimilarity = data[similarity & pattern]


In [223]:
data.shape

(17203, 20)

In [224]:
for doc in docs[100:200]:
    if doc._.date:
        displacy.render(doc,style = 'ent')

In [225]:
condition1 = np.array(['AGE_LABEL' in labels for labels in [[ ent.label_ for ent in doc.ents] for doc in docs]])

In [226]:
condition2 =  np.array(['DATE' not in labels for labels in [[ ent.label_ for ent in doc.ents] for doc in docs]])

In [227]:
data['docs'] = docs

In [228]:
filtered = data[condition1 & condition2]

In [229]:
filtered.shape

(263, 21)

In [230]:
for doc in filtered.docs:
    displacy.render(doc,style='ent')